In [4]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('diabetes.csv')

# Scale features for better performance
scaler = StandardScaler()
X=df.drop(['Outcome'],axis=1)
y=df['Outcome']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define parameter grid (reduced size)
param_dist = {
    'C': np.logspace(-2, 2, 5),  # [0.01, 0.1, 1, 10, 100]
    'gamma': np.logspace(-3, 1, 5),  # [0.001, 0.01, 0.1, 1, 10]
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3]  # Only used if kernel='poly'
}

# Use RandomizedSearchCV for faster tuning
random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train_scaled, y_train)

# Get the best SVM model
best_svm = random_search.best_estimator_

# Print best parameters
print("Best Parameters:", random_search.best_params_)


Best Parameters: {'kernel': 'linear', 'gamma': 0.01, 'degree': 3, 'C': 1.0}


In [5]:
svm1 = SVC(kernel='linear', C=1)
svm2 = SVC(kernel='rbf', C=1, gamma=0.1)
svm3 = SVC(kernel='poly', C=1, degree=2)

# Step 3: Create a VotingClassifier ensemble
ensemble = VotingClassifier(estimators=[
    ('svm1', svm1),
    ('svm2', svm2),
    ('svm3', svm3),
    ('best_svm', best_svm)
], voting='hard')

# Train the ensemble model
ensemble.fit(X_train_scaled, y_train)

# Step 4: Make predictions and evaluate accuracy
y_pred = ensemble.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print("Ensemble Accuracy:", accuracy)

Ensemble Accuracy: 0.7337662337662337


In [6]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load dataset
df = pd.read_csv('diabetes.csv')

# Split features and target
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define hyperparameter grid for RandomizedSearchCV (broader search space)
param_dist = {
    'C': np.logspace(-2, 3, 6),  # More values for C
    'gamma': np.logspace(-3, 2, 6),  # More values for gamma
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3]  # Include higher degree for polynomial kernel
}

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=15, cv=5, n_jobs=-1, random_state=42)
random_search.fit(X_train_scaled, y_train)

# Get the best SVM model
best_svm = random_search.best_estimator_

# Print best parameters
print("Best Parameters:", random_search.best_params_)

# Define SVM classifiers with the best parameters for each kernel type
svm1 = SVC(kernel='linear', C=best_svm.C)
svm2 = SVC(kernel='rbf', C=best_svm.C, gamma=best_svm.gamma)
svm3 = SVC(kernel='poly', C=best_svm.C, degree=best_svm.degree)

# Create a VotingClassifier ensemble
ensemble = VotingClassifier(estimators=[
    ('svm1', svm1),
    ('svm2', svm2),
    ('svm3', svm3),
    ('best_svm', best_svm)
], voting='hard')

# Train the ensemble model
ensemble.fit(X_train_scaled, y_train)

# Evaluate on the test set
y_pred = ensemble.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print("Ensemble Accuracy:", accuracy)


Best Parameters: {'kernel': 'linear', 'gamma': 0.01, 'degree': 3, 'C': 1000.0}
Ensemble Accuracy: 0.7662337662337663


In [7]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load dataset
df = pd.read_csv('diabetes.csv')

# Split features and target
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define a smaller hyperparameter grid for RandomizedSearchCV
param_dist = {
    'C': np.logspace(-2, 2, 3),  # Smaller range for C
    'gamma': np.logspace(-3, 1, 2),  # Smaller range for gamma
    'kernel': ['linear', 'rbf'],  # Focus on linear and rbf for speed
    'degree': [2,3,4]  # Include lower degrees for the poly kernel
}

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist, n_iter=5, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train_scaled, y_train)

# Get the best SVM model
best_svm = random_search.best_estimator_

# Print best parameters
print("Best Parameters:", random_search.best_params_)

# Define SVM classifiers with the best parameters for each kernel type
svm1 = SVC(kernel='linear', C=best_svm.C)
svm2 = SVC(kernel='rbf', C=best_svm.C, gamma=best_svm.gamma)
svm3 = SVC(kernel='poly', C=best_svm.C, degree=best_svm.degree)

# Create a VotingClassifier ensemble
ensemble = VotingClassifier(estimators=[
    ('svm1', svm1),
    ('svm2', svm2),
    ('svm3', svm3),
    ('best_svm', best_svm)
], voting='hard')

# Train the ensemble model
ensemble.fit(X_train_scaled, y_train)

# Evaluate on the test set
y_pred = ensemble.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print("Ensemble Accuracy:", accuracy)


Best Parameters: {'kernel': 'linear', 'gamma': 0.001, 'degree': 3, 'C': 1.0}
Ensemble Accuracy: 0.7597402597402597
